In [1]:
from utils import authenticate

PROJECT_ID = "stately-command-416115"
credentials = authenticate()
REGION = "us-central1"
print(PROJECT_ID)

import vertexai
vertexai.init(project=PROJECT_ID, location=REGION, credentials=credentials)

stately-command-416115


In [3]:
import pandas as pd
so_database = pd.read_csv('so_database_app.csv')
print("Shape: " + str(so_database.shape))
print(so_database)

Shape: (2000, 3)
                                             input_text  \
0     python's inspect.getfile returns "<string>"<p>...   
1     Passing parameter to function while multithrea...   
2     How do we test a specific method written in a ...   
3     how can i remove the black bg color of an imag...   
4     How to extract each sheet within an Excel file...   
...                                                 ...   
1995  Is it possible to made inline-block elements l...   
1996  Flip Clock code works on Codepen and doesn't w...   
1997  React Native How can I put one view in front o...   
1998  setting fixed width with 100% height of the pa...   
1999  How to make sidebar button not bring viewpoint...   

                                            output_text category  
0     <p><code>&lt;string&gt;</code> means that the ...   python  
1     <p>Try this and note the difference:</p>\n<pre...   python  
2     <p>Duplicate of <a href="https://stackoverflow...   python  
3     

In [5]:
from vertexai.language_models import TextEmbeddingModel
embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

import numpy as np
from utils import encode_text_to_embedding_batched

import pickle
with open('question_embeddings_app.pkl', 'rb') as file:
    question_embeddings = pickle.load(file)
    print(question_embeddings)

[[-0.03571156 -0.00240684  0.05860338 ... -0.03100227 -0.00855574
  -0.01997405]
 [-0.02024316 -0.0026255   0.01940405 ... -0.02158143 -0.05655403
  -0.01040497]
 [-0.05175979 -0.03712264  0.02699278 ... -0.07055898 -0.0402537
   0.00092099]
 ...
 [-0.00580394 -0.01621097  0.05829635 ... -0.03350992 -0.05343556
  -0.06016821]
 [-0.00436622 -0.02692963  0.03363771 ... -0.01686567 -0.03812337
  -0.02329491]
 [-0.04240424 -0.01633749  0.05516777 ... -0.02697376 -0.01751165
  -0.04558187]]


In [6]:
so_database['embeddings'] = question_embeddings.tolist()
so_database

,input_text,output_text,category,embeddings
0,"python's inspect.getfile returns ""<string>""<p>...",<p><code>&lt;string&gt;</code> means that the ...,python,"[-0.03571155667304993, -0.0024068362545222044,..."
1,Passing parameter to function while multithrea...,<p>Try this and note the difference:</p>\n<pre...,python,"[-0.020243162289261818, -0.002625499852001667,..."
2,How do we test a specific method written in a ...,"<p>Duplicate of <a href=""https://stackoverflow...",python,"[-0.05175979062914848, -0.03712264448404312, 0..."
3,how can i remove the black bg color of an imag...,<p>The alpha channel &quot;disappears&quot; be...,python,"[0.02206624671816826, -0.028208276256918907, 0..."
4,How to extract each sheet within an Excel file...,<p>You need to specify the <code>index</code> ...,python,"[-0.05498068407177925, -0.0032414537854492664,..."
...,...,...,...,...
1995,Is it possible to made inline-block elements l...,<p>If this is only for the visual purpose then...,css,"[-0.009190441109240055, -0.01732615754008293, ..."
1996,Flip Clock code works on Codepen and doesn't w...,<p>You forgot to attach the CSS file for the f...,css,"[-0.009033069014549255, -0.0009270847076550126..."
1997,React Native How can I put one view in front o...,<p>You can do it using zIndex for example:</p>...,css,"[-0.005803938489407301, -0.016210969537496567,..."
1998,setting fixed width with 100% height of the pa...,<p>You can use <code>width: calc(100% - 100px)...,css,"[-0.004366223234683275, -0.02692963369190693, ..."


In [17]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances_argmin as distances_argmin

query = ['How to concate dataframes pandas']
query_embedding = embedding_model.get_embeddings(query)[0].values
cos_sim_array = cosine_similarity([query_embedding], list(so_database.embeddings.values))
cos_sim_array.shape

(1, 2000)

In [18]:
index_doc_cosine = np.argmax(cos_sim_array)
index_doc_distances = distances_argmin([query_embedding], list(so_database.embeddings.values))[0]
so_database.input_text[index_doc_cosine]

"Pandas - concating multi-indexed dataframes keeps duplicate indizes<p>Hello I'm trying to read in multiple dfs of the same structre and concating them into a single one however the combined df somehow keeps duplicates in the indizes....</p>\n<pre><code>df1 = processUSAdata(2223)\ndf2 = processUSAdata(2224)\n\nprint(df1)\nprint(df2)\n\ntest = pd.concat([df1,df2])\ntest.set_index(['state','year'],inplace=True)\nprint(test)\n</code></pre>\n<p>outputs</p>\n<pre><code>         state  population  violent_crime  ...    theft     gta  year\n0      ALABAMA     4903185         510.81  ...  1886.06  256.51  2223\n1       ALASKA      731545         867.07  ...  2066.04  357.74  2223\n2      ARIZONA     7278717         455.31  ...  1796.86  249.37  2223\n3     ARKANSAS     3017804         584.63  ...  2012.56  245.87  2223\n4   CALIFORNIA    39512223         441.21  ...  1586.35  358.77  2223\n5     COLORADO     5758736         380.95  ...  1858.26  383.99  2223\n6  CONNECTICUT     3565287        

In [19]:
so_database.output_text[index_doc_cosine]

"<p>It's probably because your index is not sorted:</p>\n<pre><code>test = pd.concat([df1, df2]).set_index(['state', 'year']).sort_index()\nprint(test)\n\n# Output\n                  population  violent_crime    theft     gta\nstate       year                                            \nALABAMA     2223     4903185         510.81  1886.06  256.51\n            2224     4903185         510.81  1886.06  256.51\nALASKA      2223      731545         867.07  2066.04  357.74\n            2224      731545         867.07  2066.04  357.74\nARIZONA     2223     7278717         455.31  1796.86  249.37\n            2224     7278717         455.31  1796.86  249.37\nARKANSAS    2223     3017804         584.63  2012.56  245.87\n            2224     3017804         584.63  2012.56  245.87\nCALIFORNIA  2223    39512223         441.21  1586.35  358.77\n            2224    39512223         441.21  1586.35  358.77\nCOLORADO    2223     5758736         380.95  1858.26  383.99\n            2224     5758736 

In [20]:
from vertexai.language_models import TextGenerationModel
generation_model = TextGenerationModel.from_pretrained('text-bison@001')
context = "Question: " + so_database.input_text[index_doc_cosine] + \
"\n Answer: " + so_database.output_text[index_doc_cosine]

prompt = f"""Here is the context: {context}
Using the relevant information from the context, provide an answer to teh query: {query}."
If the context doesn't provide any relevant information, answer with [I couldn't find a good match in the document database for your query]"""

from IPython.display import Markdown, display

t_value = 0.2
response = generation_model.predict(prompt=prompt, temperature=t_value, max_output_tokens=1024)
display(Markdown(response.text))

The answer is to sort the index before concatenating the dataframes.

In [21]:
from sklearn.metrics.pairwise import cosine_similarity
query = ['How to make the perfect lasagna']
query_embedding = embedding_model.get_embeddings(query)[0].values
cos_sim_array = cosine_similarity([query_embedding], list(so_database.embeddings.values))
cos_sim_array

array([[0.52794666, 0.49346994, 0.50784253, ..., 0.45782265, 0.5483748 ,
        0.49693718]])

In [22]:
index_doc = np.argmax(cos_sim_array)
context = so_database.input_text[index_doc] + "\n Answer: " + so_database.output_text[index_doc]
prompt = f"""Here is the context: {context}
             Using the relevant information from the context,
             provide an answer to the query: {query}."
             If the context doesn't provide \
             any relevant information, answer with 
             [I couldn't find a good match in the \
             document database for your query]
             """
t_value = 0.2
response = generation_model.predict(prompt = prompt,
                                    temperature = t_value,
                                    max_output_tokens = 1024)
display(Markdown(response.text))

I couldn't find a good match in the document database for your query.

In [24]:
import scann # I wasn't immediately able to import this in Windows, so I stopped here
from utils import create_index

index = create_index(embedded_dataset = question_embeddings,
                     num_leaves = 25, 
                     num_leaves_to_search = 10,
                     training_sample_size = 2000)

query = "how to concate dataframes pandas"

ImportError: cannot import name 'create_index' from 'utils' (c:\Users\RKeelan\Src\deep-learning-ai\utils.py)